Denne fil skal bruges til at hente og indlæse data.
Det kan fx være fra en CSV-fil, Excel-fil, database eller et API.

Målet her er bare at få data ind i programmet som et pandas DataFrame,
så vi senere kan rense og analysere det.